In [1]:
# from __future__ import print_statement
# import time
# pip install swagger_client
# import swagger_client
# from swagger_client.rest import ApiException
# from pprint import pprint

import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
import json

import ipywidgets as widgets
import spacy
import math
import numpy as np
import scattertext as st
from sklearn import manifold
from sklearn.metrics import euclidean_distances

In [2]:
# pip install json_normalize
# dir(requests.get("https://api.nal.usda.gov/fdc/v1/food/534358?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ"))
# food = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/food/534358?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ").content)
# for item in food.items():
#     print(item)

# Direct ID pulls

In [6]:
food = pd.read_csv('complementaryFiles/food.csv')

In [7]:
food.fillna('N/A',inplace=True)

In [11]:
food[(food['description'].str.contains('salmon'))&(food['description'].str.contains('Alaska')&(food['description'].str.contains('sockeye')))]

salmonLst = list(food[food['description'].str.contains('salmon')]['fdc_id'])


In [12]:
# json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=167614,167615&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)

In [13]:
salmonLst = str(salmonLst).replace('[','').replace(']','')

In [14]:
apiKey = 'IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ'
# salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=167614,167615&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)
salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds={salmonLst}&api_key={apiKey}').content)

#survey food
# salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=1098960&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)

df = pd.DataFrame(salmon)
df = df.dropna(subset=['foodPortions'])
df = df.reset_index()
df = df.drop('index',axis=1)
df.head()

,fdcId,description,publicationDate,foodNutrients,dataType,foodClass,scientificName,foodComponents,foodAttributes,nutrientConversionFactors,inputFoods,ndbNumber,isHistoricalReference,foodCategory,foodPortions,wweiaFoodCategory,foodCode,startDate,endDate
0,171985,"Fish, salmon, chinook, smoked, (lox), regular",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14494, 'proteinValue': 4.27, 'fatValue...",[],15179.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89683, 'gramWeight': 85.0, 'sequenceNu...",NaN,NaN,NaN,NaN
1,171998,"Fish, salmon, Atlantic, wild, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14507, 'proteinValue': 4.27, 'fatValue...",[],15209.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89708, 'gramWeight': 85.0, 'sequenceNu...",NaN,NaN,NaN,NaN
2,171999,"Fish, salmon, chinook, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14508, 'proteinValue': 4.27, 'fatValue...",[],15210.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89711, 'gramWeight': 154.0, 'sequenceN...",NaN,NaN,NaN,NaN
3,172000,"Fish, salmon, chum, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14509, 'proteinValue': 4.27, 'fatValue...",[],15211.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89713, 'gramWeight': 154.0, 'sequenceN...",NaN,NaN,NaN,NaN
4,172001,"Fish, salmon, pink, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14510, 'proteinValue': 4.27, 'fatValue...",[],15212.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89715, 'gramWeight': 124.0, 'sequenceN...",NaN,NaN,NaN,NaN


In [15]:
current_food = df['description'].values[0]
df['foodNutrients'][0] #hidden dataframe with all of the nutrient data
macros = ['Protein','Carbohydrates','Fat','Total lipid (fat)']
json_normalize(df['nutrientConversionFactors'][0][0])
json_normalize(df['foodCategory'][0])
len(list(json_normalize(df['foodNutrients'][0]).columns))

#legacy food iteration
for x in range(df.shape[0]):
    col_len = len(list(json_normalize(df['foodNutrients'][x]).columns))
    if col_len == 17:
        nutrient_df = json_normalize(df['foodNutrients'][x])
        # display(nutrient_df)
    else:
        pass

#survey food
nutrient_df = json_normalize(df['foodNutrients'][0])
macro_df = nutrient_df[nutrient_df['nutrient.name'].isin(macros)]
conv_factors = json_normalize(df['nutrientConversionFactors'][0])
portion_df = json_normalize(df['foodPortions'][0])
input_df = json_normalize(df['inputFoods'][0])
protein = macro_df[macro_df['nutrient.name']=='Protein']['amount'].values[0]
calories = nutrient_df[nutrient_df['nutrient.unitName']=='kcal']['amount'].values[0]
portion_grams = portion_df['gramWeight'].values[0]
# portion_ounces = portion_df[portion_df['portionDescription'].str.contains('oz')]['portionDescription'].values[0].split()[0]
print(f'Data for {current_food}')
print('===================')
print('===================')
print('Protein per serving')
print(protein)
print('===================')
print('Calories per serving')
print(calories)

Data for Fish, salmon, chinook, smoked, (lox), regular
Protein per serving
18.28
Calories per serving
117.0


# PDF generation

In [16]:
# pip install reportlab

In [19]:
#table gen - https://pythonguides.com/create-and-modify-pdf-file-in-python/

from reportlab.pdfgen import canvas
# c = canvas.Canvas("salmon.pdf")
from reportlab.lib import colors  
from reportlab.lib.pagesizes import letter, inch  
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle   
# # move the origin up and to the left
# c.translate(inch,inch)
# c.setFont("Helvetica", 20)
# c.setFillColorRGB(0,0,0)
# c.drawString(.5*inch, 9*inch, f'Data for {current_food}')
# c.drawString(.5*inch, 8*inch, "========================")
# c.drawString(.5*inch, 7*inch, "========================")
# c.drawString(.5*inch, 6*inch, f'Protein per serving = {protein}')
# c.drawString(.5*inch, 5*inch, "========================")
# c.drawString(.5*inch, 4*inch, f'Calories per serving = {calories}')
# c.drawString(.5*inch, 3*inch, "========================")

# creating a pdf file to add tables  
my_doc = SimpleDocTemplate("salmon.pdf", pagesize = letter)  
my_obj = []  
# defining Data to be stored on table  
my_data = [  
   ["Food", "Calories", "Protein"],  
   [current_food, calories, protein],
]  
# Creating the table with 5 rows  
my_table = Table(my_data, 1 * [2.5 * inch], 2 * [0.5 * inch])  
# setting up style and alignments of borders and grids  
my_table.setStyle(  
   TableStyle(  
       [  
           ("ALIGN", (1, 1), (0, 0), "LEFT"),  
           ("VALIGN", (-1, -1), (-1, -1), "TOP"),  
           ("ALIGN", (-1, -1), (-1, -1), "RIGHT"),  
           ("VALIGN", (-1, -1), (-1, -1), "TOP"),  
           ("INNERGRID", (0, 0), (-1, -1), 1, colors.gray),  
        #    ("BOX", (0, 0), (-1, -1), 2, colors.black),  
       ]  
   )  
)  
my_obj.append(my_table)  
my_doc.build(my_obj)

# c.showPage()
# c.save()

## Search API functionality

In [78]:
cheddar = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/foods/search?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ&query=Cheddar%20Cheese").content)
cheese = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/foods/search?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ&query=Cheese").content)

In [79]:
for key, value in cheese.items():
    if key == 'foods':
#         ched = pd.DataFrame(columns = ['fdcId', 'description', 'lowercaseDescription', 'dataType', 'gtinUpc','publishedDate', 
#         'brandOwner', 'brandName', 'ingredients','marketCountry', 'foodCategory', 'allHighlightFields', 'score','foodNutrients'])
        df=[]
        for val in value:
#             for key,value in val.items():
#                 print(key,value)
            df.append(pd.DataFrame(val))
    else:
        pass

In [80]:
df = pd.concat(df,sort=False)

In [81]:
df

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,foodCategory,allHighlightFields,score,foodNutrients,commonNames,additionalDescriptions,foodCode,ndbNumber
0,1912989,CHEESE,cheese,Branded,850276005014,2021-07-29,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,"<b>Ingredients</b>: CHEDDAR <em>CHEESE</em>, (...",607.71750,"{'nutrientId': 1003, 'nutrientName': 'Protein'...",NaN,NaN,NaN,NaN
1,1912989,CHEESE,cheese,Branded,850276005014,2021-07-29,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,"<b>Ingredients</b>: CHEDDAR <em>CHEESE</em>, (...",607.71750,"{'nutrientId': 1004, 'nutrientName': 'Total li...",NaN,NaN,NaN,NaN
2,1912989,CHEESE,cheese,Branded,850276005014,2021-07-29,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,"<b>Ingredients</b>: CHEDDAR <em>CHEESE</em>, (...",607.71750,"{'nutrientId': 1005, 'nutrientName': 'Carbohyd...",NaN,NaN,NaN,NaN
3,1912989,CHEESE,cheese,Branded,850276005014,2021-07-29,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,"<b>Ingredients</b>: CHEDDAR <em>CHEESE</em>, (...",607.71750,"{'nutrientId': 1008, 'nutrientName': 'Energy',...",NaN,NaN,NaN,NaN
4,1912989,CHEESE,cheese,Branded,850276005014,2021-07-29,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,"<b>Ingredients</b>: CHEDDAR <em>CHEESE</em>, (...",607.71750,"{'nutrientId': 2000, 'nutrientName': 'Sugars, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1098013,"Cheese, Fontina","cheese, fontina",Survey (FNDDS),NaN,2020-10-30,NaN,NaN,NaN,NaN,Cheese,,188.67055,"{'nutrientId': 1278, 'nutrientName': 'PUFA 2:5...",,,14104600.0,NaN
61,1098013,"Cheese, Fontina","cheese, fontina",Survey (FNDDS),NaN,2020-10-30,NaN,NaN,NaN,NaN,Cheese,,188.67055,"{'nutrientId': 1279, 'nutrientName': 'MUFA 22:...",,,14104600.0,NaN
62,1098013,"Cheese, Fontina","cheese, fontina",Survey (FNDDS),NaN,2020-10-30,NaN,NaN,NaN,NaN,Cheese,,188.67055,"{'nutrientId': 1280, 'nutrientName': 'PUFA 22:...",,,14104600.0,NaN
63,1098013,"Cheese, Fontina","cheese, fontina",Survey (FNDDS),NaN,2020-10-30,NaN,NaN,NaN,NaN,Cheese,,188.67055,"{'nutrientId': 1292, 'nutrientName': 'Fatty ac...",,,14104600.0,NaN


In [96]:
from pandas.io.json import json_normalize
nutrients = json_normalize(df['foodNutrients'])

In [97]:
nutrients[nutrients['nutrientName']=='Protein']

,nutrientId,nutrientName,nutrientNumber,unitName,derivationCode,derivationDescription,value
0,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,32.10
14,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,14.30
27,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,16.80
40,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,17.90
53,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,21.40
66,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,21.40
80,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,25.00
94,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,21.40
107,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,5.17
121,1003,Protein,203,G,LCCS,Calculated from value per serving size measure,21.10


In [100]:
df[df['fdcId']==1497465]['foodNutrients']

0     {'nutrientId': 1003, 'nutrientName': 'Protein'...
1     {'nutrientId': 1004, 'nutrientName': 'Total li...
2     {'nutrientId': 1005, 'nutrientName': 'Carbohyd...
3     {'nutrientId': 1008, 'nutrientName': 'Energy',...
4     {'nutrientId': 2000, 'nutrientName': 'Sugars, ...
5     {'nutrientId': 1079, 'nutrientName': 'Fiber, t...
6     {'nutrientId': 1087, 'nutrientName': 'Calcium,...
7     {'nutrientId': 1089, 'nutrientName': 'Iron, Fe...
8     {'nutrientId': 1093, 'nutrientName': 'Sodium, ...
9     {'nutrientId': 1104, 'nutrientName': 'Vitamin ...
10    {'nutrientId': 1162, 'nutrientName': 'Vitamin ...
11    {'nutrientId': 1253, 'nutrientName': 'Choleste...
12    {'nutrientId': 1257, 'nutrientName': 'Fatty ac...
13    {'nutrientId': 1258, 'nutrientName': 'Fatty ac...
Name: foodNutrients, dtype: object

In [ ]:
#### lets use k nearest neighbors on macros !!!! ######